In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim 
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import numpy as np
from tqdm import tqdm 

In [2]:
#Check device 
if torch.backends.mps.is_available():
    device = torch.device('mps')

In [11]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import ssl

def get_dataloaders(batch_size=128, data_dir='./data'):
    # Fix SSL certificate issue
    ssl._create_default_https_context = ssl._create_unverified_context
    
    # Define transforms
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
    ])
    
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
    ])
    
    # Download CIFAR-100
    trainset = datasets.CIFAR100(
        root=data_dir, train=True, download=True, transform=transform_train
    )
    testset = datasets.CIFAR100(
        root=data_dir, train=False, download=True, transform=transform_test
    )
    
    trainloader = DataLoader(
        trainset, batch_size=batch_size, shuffle=True, num_workers=2
    )
    testloader = DataLoader(
        testset, batch_size=batch_size, shuffle=False, num_workers=2
    )
    
    return trainloader, testloader

In [12]:
# Model A: VGG without BatchNorm

class VGG_NoBN(nn.Module):
    def __init__(self, num_classes = 100):
        super(VGG_NoBN, self).__init__()
        
        self.features = nn.Sequential(
            #Block 1
            nn.Conv2d(3, 64, 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.Conv2d(64, 64, 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(2, 2),

            #Block 2
            nn.Conv2d(64, 128, 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.Conv2d(128, 128, 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(2,2),

            #Block3
            nn.Conv2d(128, 256, 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.Conv2d(256, 256, 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.Conv2d(256, 256, 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(2,2),

            # Block 4
            nn.Conv2d(256, 512, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(512 * 2 * 2, 512),
            nn.ReLU(inplace = True),
            nn.Dropout(0.5),
            nn.Linear(512, 512),
            nn.ReLU(inplace = True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal(m.weight, mode = 'fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


            


In [13]:
# Model b: VGG with BatchNorm before activation
class VGG_BN_Before(nn.Module):
    def __init__(self, num_classes=100):
        super(VGG_BN_Before, self).__init__()
        
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            # Block 2
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            # Block 3
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            # Block 4
            nn.Conv2d(256, 512, 3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(512 * 2 * 2, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
        
        self._initialize_weights()
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


In [14]:
# Model c: VGG with BatchNorm after activation
class VGG_BN_After(nn.Module):
    def __init__(self, num_classes=100):
        super(VGG_BN_After, self).__init__()
        
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2, 2),
            
            # Block 2
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2, 2),
            
            # Block 3
            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2, 2),
            
            # Block 4
            nn.Conv2d(256, 512, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(512),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(512),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(512),
            nn.MaxPool2d(2, 2),
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(512 * 2 * 2, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
        
        self._initialize_weights()
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


In [15]:
#Training function

def train_epoch(model, trainloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(trainloader, desc = 'Training')
    for inputs, targets in pbar:
        inputs, targets = inputs.to(device), targets.to(device)

        #Training steps

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        #Tracking metrics
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        # Update progress bar with current loss and accuracy
        pbar.set_postfix({'loss': running_loss/len(pbar), 
                         'acc': 100.*correct/total})
    # Return average loss and accuracy for the epoch
    return running_loss / len(trainloader), 100. * correct / total
        


In [16]:
# Testing function with top-1 and top-5 accuracy
def test(model, testloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct_top1 = 0
    correct_top5 = 0
    total = 0
    
    with torch.no_grad():
        pbar = tqdm(testloader, desc='Testing')
        for inputs, targets in pbar:
            inputs, targets = inputs.to(device), targets.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            running_loss += loss.item()
            
            # Top-1 accuracy
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct_top1 += predicted.eq(targets).sum().item()
            
            # Top-5 accuracy
            _, top5_pred = outputs.topk(5, 1, True, True)
            top5_pred = top5_pred.t()
            correct_top5 += top5_pred.eq(targets.view(1, -1).expand_as(top5_pred)).sum().item()
            
            pbar.set_postfix({'loss': running_loss/len(pbar),
                            'top1': 100.*correct_top1/total,
                            'top5': 100.*correct_top5/total})
    
    top1_acc = 100. * correct_top1 / total
    top5_acc = 100. * correct_top5 / total
    avg_loss = running_loss / len(testloader)
    
    return avg_loss, top1_acc, top5_acc



In [17]:
# Main training function
def train_model(model, model_name, trainloader, testloader, num_epochs=30):
    print(f"\n{'='*60}")
    print(f"Training {model_name}")
    print(f"{'='*60}")
    
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    #optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30, 40], gamma=0.1)
    
    best_top1_acc = 0.0
    best_top5_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print(f"Learning rate: {scheduler.get_last_lr()[0]:.6f}")
        
        train_loss, train_acc = train_epoch(model, trainloader, criterion, optimizer, device)
        test_loss, top1_acc, top5_acc = test(model, testloader, criterion, device)
        
        scheduler.step()
        
        if top1_acc > best_top1_acc:
            best_top1_acc = top1_acc
            best_top5_acc = top5_acc
        
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"Test Loss: {test_loss:.4f}, Top-1 Acc: {top1_acc:.2f}%, Top-5 Acc: {top5_acc:.2f}%")
        print(f"Best Top-1: {best_top1_acc:.2f}%, Best Top-5: {best_top5_acc:.2f}%")
    
    return best_top1_acc, best_top5_acc

In [18]:
# Main execution
if __name__ == "__main__":
    # Prepare data
    trainloader, testloader = get_dataloaders(batch_size=128)
    
    # Store results
    results = {}
    
    # Train model a: VGG without BatchNorm
    print("\n" + "="*60)
    print("MODEL A: VGG WITHOUT BATCH NORMALIZATION")
    print("="*60)
    model_a = VGG_NoBN()
    top1_a, top5_a = train_model(model_a, "VGG without BatchNorm", 
                                  trainloader, testloader, num_epochs=30)
    results['No BatchNorm'] = {'top1': top1_a, 'top5': top5_a}
    
    # Train model b: VGG with BatchNorm before activation
    print("\n" + "="*60)
    print("MODEL B: VGG WITH BATCH NORMALIZATION BEFORE ACTIVATION")
    print("="*60)
    model_b = VGG_BN_Before()
    top1_b, top5_b = train_model(model_b, "VGG with BatchNorm before ReLU", 
                                  trainloader, testloader, num_epochs=30)
    results['BatchNorm Before ReLU'] = {'top1': top1_b, 'top5': top5_b}
    
    # Train model c: VGG with BatchNorm after activation
    print("\n" + "="*60)
    print("MODEL C: VGG WITH BATCH NORMALIZATION AFTER ACTIVATION")
    print("="*60)
    model_c = VGG_BN_After()
    top1_c, top5_c = train_model(model_c, "VGG with BatchNorm after ReLU", 
                                  trainloader, testloader, num_epochs=30)
    results['BatchNorm After ReLU'] = {'top1': top1_c, 'top5': top5_c}
    
    # Print final comparison
    print("\n" + "="*60)
    print("FINAL RESULTS COMPARISON")
    print("="*60)
    print(f"{'Model':<30} {'Top-1 Accuracy':<20} {'Top-5 Accuracy':<20}")
    print("-" * 70)
    for model_name, acc in results.items():
        print(f"{model_name:<30} {acc['top1']:>18.2f}% {acc['top5']:>18.2f}%")
    print("="*60)

100%|██████████| 169M/169M [00:48<00:00, 3.46MB/s] 
/var/folders/80/t054jgpx5tqd5k22zqq21r4m0000gn/T/ipykernel_21232/3741677396.py:61: FutureWarning: `nn.init.kaiming_normal` is now deprecated in favor of `nn.init.kaiming_normal_`.
  nn.init.kaiming_normal(m.weight, mode = 'fan_out', nonlinearity='relu')



MODEL A: VGG WITHOUT BATCH NORMALIZATION

Training VGG without BatchNorm

Epoch 1/30
Learning rate: 0.001000


Testing: 100%|██████████| 79/79 [00:13<00:00,  5.70it/s, loss=4.61, top1=1, top5=5]        


Train Loss: 4.6058, Train Acc: 0.86%
Test Loss: 4.6052, Top-1 Acc: 1.00%, Top-5 Acc: 5.00%
Best Top-1: 1.00%, Best Top-5: 5.00%

Epoch 2/30
Learning rate: 0.001000


Testing: 100%|██████████| 79/79 [00:13<00:00,  5.73it/s, loss=4.61, top1=1, top5=5]        


...

Train Loss: 1.5497, Train Acc: 57.28%
Test Loss: 1.6092, Top-1 Acc: 56.31%, Top-5 Acc: 83.85%
Best Top-1: 56.31%, Best Top-5: 83.85%

Epoch 29/30
Learning rate: 0.001000


Testing: 100%|██████████| 79/79 [00:13<00:00,  5.65it/s, loss=1.6, top1=56.4, top5=83.7]  


Train Loss: 1.5313, Train Acc: 57.74%
Test Loss: 1.6033, Top-1 Acc: 56.42%, Top-5 Acc: 83.69%
Best Top-1: 56.42%, Best Top-5: 83.69%

Epoch 30/30
Learning rate: 0.001000


Testing: 100%|██████████| 79/79 [00:14<00:00,  5.64it/s, loss=1.59, top1=56.6, top5=83.8] 

Train Loss: 1.5176, Train Acc: 58.00%
Test Loss: 1.5883, Top-1 Acc: 56.64%, Top-5 Acc: 83.81%
Best Top-1: 56.64%, Best Top-5: 83.81%

FINAL RESULTS COMPARISON
Model                          Top-1 Accuracy       Top-5 Accuracy      
----------------------------------------------------------------------
No BatchNorm                                 1.00%               5.00%
BatchNorm Before ReLU                       48.16%              79.88%
BatchNorm After ReLU                        56.64%              83.81%
